# PicTex Text Detection Model with Zach

From meeting with Zach on 8/10/20 because `output tensors` weren't 1's and 0's


### 1. Create labels

In [57]:
from collections import OrderedDict
import numpy as np
import torch
import os

data_dir = "./final/"

classes = os.listdir(data_dir)
num_classes = len(classes)

classes_encode, classes_decode = {}, {}
for i, name in enumerate(classes):
    classes_encode[name] = i
    classes_decode[i] = name

encode_dict, decode_dict = OrderedDict(classes_encode), OrderedDict(classes_encode)

print(f"There are {num_classes} classes")
print(classes_encode)
print(classes_decode)

There are 73 classes
{'(': 0, ')': 1, '+': 2, '-': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '=': 14, 'a': 15, 'alpha': 16, 'ast': 17, 'b': 18, 'beta': 19, 'c': 20, 'comma': 21, 'd': 22, 'delta': 23, 'e': 24, 'emptyset': 25, 'f': 26, 'forall': 27, 'full_stop': 28, 'g': 29, 'greater': 30, 'h': 31, 'implies': 32, 'in': 33, 'infty': 34, 'int': 35, 'j': 36, 'k': 37, 'l': 38, 'lambda': 39, 'land': 40, 'leq': 41, 'lesser': 42, 'm': 43, 'mu': 44, 'n': 45, 'nabla': 46, 'Naturals': 47, 'neq': 48, 'o': 49, 'p': 50, 'perp': 51, 'pi': 52, 'q': 53, 'r': 54, 'Reals': 55, 's': 56, 'setminus': 57, 'sigma': 58, 'sim': 59, 'sum': 60, 'supset': 61, 't': 62, 'theta': 63, 'u': 64, 'v': 65, 'varepsilon': 66, 'w': 67, 'x': 68, 'y': 69, 'z': 70, '[': 71, ']': 72}
{0: '(', 1: ')', 2: '+', 3: '-', 4: '0', 5: '1', 6: '2', 7: '3', 8: '4', 9: '5', 10: '6', 11: '7', 12: '8', 13: '9', 14: '=', 15: 'a', 16: 'alpha', 17: 'ast', 18: 'b', 19: 'beta', 20: 'c', 21: 'comma', 22:

### 2. Create the `Dataset` and `Dataloader` objects

In [86]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data import random_split
import numpy as np
import torch
from PIL import Image

# Choose either colored / grayscale
num_output_channels = 1
normal = (0.5,) # uncomment this for grayscale
# normal = (0.5, 0.5, 0.5) # uncomment this for colored

# choose batch size
batch_size = 64

transform = transforms.Compose(
    [transforms.Grayscale(num_output_channels),
     transforms.Resize((32, 32)), 
     transforms.ToTensor(),
     transforms.Normalize(normal, normal)])

class PicTexDataset(Dataset):
    def __init__(self, root_dir, encode_dict, num_classes, transform=None):
        """
        Args:
            root_dir (string): Directory containing images sorted by class folders
            encode_dict (Ordered dict): Dictionary with class names zipped from 0-(num_classes-1)
            num_classes (int): Number of classes (SHOULD EQUAL LEN of ENCODE_DICT)
            transform (torchvision.transforms): Transforms to be applied to images 
        """
        self.root_dir = root_dir
        self.encode_dict = encode_dict
        self.num_classes = num_classes
        self.transform = transform
        
        """
        Loading images:
            all_paths (string list): Path of every image
            all_paths_class (string list): Bijection with all_paths. Class name for each path
            all_images (string * string list): List of these objects... (class name, path of image)
        """
        all_paths, all_paths_class = [], []
        for name in encode_dict.keys():
            list_classes = os.listdir(root_dir + name)
            all_paths += list(map(lambda s : root_dir + name + "/" + s, list_classes))
            all_paths_class += [name] * len(list_classes)
        
        self.all_images = list(zip(all_paths_class, all_paths))

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        img_class, img_name = self.all_images[idx]
        try:
            image = Image.open(img_name)
            if self.transform:
                image = self.transform(image)
        except OSError:
            print(img_name, img_class)
        # image = image.unsqueeze(1)
        
        #label = torch.zeros(self.num_classes)
        #label[self.encode_dict[img_class]] = 1 
        label = self.encode_dict[img_class]
        
        return image, label
    
def load_split_train_test(datadir, valid_size = .2):
    dataset = PicTexDataset(data_dir, encode_dict, num_classes, transform)
    num_test = int(valid_size*len(dataset))
    num_train = len(dataset) - num_test
    
    train_data, test_data = random_split(dataset, (num_train, num_test))
    
    trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)
    return trainloader, testloader

trainloader, testloader = load_split_train_test(data_dir, .15)


### 3. Test `Dataloader`

In [82]:
for image, label in trainloader:
    print(f"Label: {label}")

Label: tensor([30, 44, 32, 13, 42, 10,  7, 33, 17, 20,  3, 70, 42, 62, 28, 52])
Label: tensor([ 0,  3, 24,  4, 19,  6, 72, 58, 42,  5, 15, 22, 39,  3, 21, 18])
Label: tensor([17, 12, 58, 72, 26, 71, 54, 25, 60, 68, 45, 16, 47, 18, 33, 44])
Label: tensor([ 8, 54, 13,  3,  3, 49, 59,  6, 70, 23,  7, 58, 26, 61, 70,  7])
Label: tensor([23, 72, 45, 40,  5, 37, 21, 63, 33, 50, 25, 14, 43, 49, 17, 38])
Label: tensor([33, 60, 17,  3, 61, 27, 58, 56, 72,  4, 37, 40, 11,  4, 26, 45])
Label: tensor([64, 13, 47, 25, 42, 10,  2, 54, 14, 11, 58,  5, 34, 62, 45, 62])
Label: tensor([ 6, 45, 69, 24, 72,  5, 67, 30, 34, 29, 43, 43, 63, 52,  7,  4])
Label: tensor([61, 11, 54, 42, 31, 34, 59, 70, 63, 45, 32, 44, 42, 47,  4, 17])
Label: tensor([ 2, 15, 44, 19, 34, 11, 51,  9, 34, 61, 67, 55, 49, 68, 58, 13])
Label: tensor([27, 21, 27, 62,  2, 34, 26, 55, 11,  2, 42, 72,  7, 10, 24, 57])
Label: tensor([70, 65, 40, 66, 70, 63, 32, 22, 22, 44, 54, 53,  5, 51, 30, 27])
Label: tensor([67,  4, 65, 42, 68, 62, 6

Label: tensor([38, 41, 36, 33, 66, 34, 37, 47, 43, 37,  7,  2, 21, 64, 23, 63])
Label: tensor([26,  6, 63, 45, 20, 61, 46, 46, 63, 26, 10, 15, 63, 26, 15, 21])
Label: tensor([21,  6, 19, 67, 22, 15,  3,  3, 37, 15, 68,  7, 56, 62,  6,  5])
Label: tensor([61, 44, 40, 70, 10, 22, 20, 52, 22, 29, 60, 71, 17, 38, 66, 62])
Label: tensor([10, 66, 52,  5,  7, 66, 34, 30,  9, 70, 21, 20, 19, 50, 61, 56])
Label: tensor([ 1, 53, 44, 61, 58, 65, 17, 46, 48, 34, 46, 52, 61, 42, 25, 20])
Label: tensor([ 5, 51,  0, 40, 26, 47, 31, 50, 49, 23,  9, 58, 10, 31,  5,  8])
Label: tensor([ 3, 24, 24, 51,  4, 61, 43, 31,  7, 22, 17, 66, 33, 27,  3, 63])
Label: tensor([ 0, 62,  7,  3, 43, 11,  6, 19, 32, 42, 72, 23, 35, 50,  5, 34])
Label: tensor([64, 49, 60, 46, 22, 64, 15, 47,  9, 10,  8, 36, 49, 19, 22,  0])
Label: tensor([20, 56, 63, 46, 56, 40, 31, 41, 33, 42, 71, 29,  5, 36, 46, 52])
Label: tensor([41, 66, 55, 32, 17, 47, 33, 70, 21, 39, 14, 26, 33, 42, 13, 16])
Label: tensor([43, 57, 13,  7, 11,  2, 3

Label: tensor([57, 47, 22, 31, 57, 31, 54, 53, 52,  8, 27, 13, 20, 53, 24, 39])
Label: tensor([49, 46, 11,  5, 25, 46, 62,  0, 17, 27, 37, 50, 13, 54, 48,  7])
Label: tensor([56, 70,  4,  0, 65, 69, 32,  1, 65, 58, 10,  9, 21, 68,  9, 17])
Label: tensor([34, 52,  9, 50, 47,  2, 24, 16, 47, 26,  0, 26, 70, 30, 33, 35])
Label: tensor([15, 30, 38, 51,  4,  7,  5, 38, 34, 17,  7,  2, 47, 14, 67,  3])
Label: tensor([59, 47, 71, 55,  9, 27,  4, 12, 51, 31, 65,  2, 19, 69, 51, 67])
Label: tensor([62, 59,  6, 32, 65, 32, 31, 54, 52, 51,  7, 67, 34, 34, 36, 64])
Label: tensor([51, 61, 19, 12, 36, 32, 14, 12, 57, 35, 50, 72, 64, 38, 30, 60])
Label: tensor([20, 48, 64, 64, 12, 47, 44, 13, 56, 21, 32, 19, 69, 20, 69, 52])
Label: tensor([18, 70, 31, 38, 40, 56, 19, 52,  4,  4, 16, 50, 18, 57,  6, 33])
Label: tensor([16, 31, 46,  1,  1, 21, 35, 34, 48, 25, 57, 42, 16, 27,  1,  0])
Label: tensor([51, 58, 57,  2, 29, 30,  8, 14,  7, 32, 70, 39,  8,  3,  1, 23])
Label: tensor([21,  4, 39, 39, 17, 12, 4

### 4. Everything else from before:

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv11 = nn.Conv2d(in_channels=num_output_channels, out_channels=20, kernel_size=5, padding=2)
        self.conv12 = nn.Conv2d(20, 20, 5, padding=2)
        self.batch1 = nn.BatchNorm2d(20)
        self.pool1  = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv21 = nn.Conv2d(20, 40, 5, padding=2)
        self.conv22 = nn.Conv2d(40, 40, 5, padding=2)
        self.batch2 = nn.BatchNorm2d(40)
        self.pool2  = nn.MaxPool2d(2,2)
        
        # Transitioning from Conv ===> Linear
        # 16 is the number of output channels in the previous conv layer.
        
        self.fc1 = nn.Linear(40 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(classes))
        self.dropconv = nn.Dropout(0.2)
        self.dropfc = nn.Dropout(0.4)
        self.soft = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = self.batch1(x)
        x = self.pool1(x)
        x = F.relu(self.conv21(x))
        x = F.relu(self.conv22(x))
        x = self.batch2(x)
        x = self.pool2(x)
        x = x.view(-1, 40 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.dropfc(x)
        x = self.soft(x)
        return x

# init the class 
model = Net()
print(model)
model.eval()
inp = torch.randn((1,1,32,32))
out = model(inp)

#model.load_state_dict(torch.load('./models/custom_label_1000.pt'))
model.train()
import torch.optim as optim

# set parameters
learning_rate = 0.001 #0.005
momentum = 0.9

def loss_optim():
    criterion = nn.CrossEntropyLoss()
    # criterion = nn.MSELoss()
    #criterion = nn.NLLLoss()
    # Adam Max
    #optimizer = torch.optim.Adamax(model.parameters(), learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    # Adam
    optimizer = optim.Adam(model.parameters(), learning_rate, amsgrad = True)
    # SGD. Momentum = remembering previous estimation change for a param (delta W)
    #optimizer = optim.SGD(model.parameters(), learning_rate, momentum)
    return criterion, optimizer

criterion, optimizer = loss_optim()
print(criterion)
print(optimizer)

%matplotlib inline
import matplotlib.pyplot as plt
from torch.autograd import Variable


def train_network():
    
    # Choose parameters
    num_epoch = 100
    mini_batch = 10 # previously batch_size
    train_losses, test_losses = [], []
    running_loss = 0
    for epoch in range(num_epoch):  # loop over the dataset multiple times
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = Variable(inputs), Variable(labels)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            # convert to float bc softmax doesn't work with long
            inputs = torch.tensor(inputs, dtype=torch.float)
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # print statistics
            if i % mini_batch == mini_batch-1:    # print every # of mini-batches
                test_loss = 0
                accuracy = 0
                # begin evaluation of validation loss
                model.eval()
                with torch.no_grad():
                    for inputs_test, labels_test in testloader:
                        if labels_test.size()[0] == batch_size:
                            ps = model.forward(inputs_test)
                            batch_loss = criterion(ps, labels_test)
                            test_loss += batch_loss.item()
                            top_p, top_class = ps.topk(1, dim=1)
                            equals = top_class == labels_test.view(*top_class.shape)
                            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                train_losses.append(running_loss/len(trainloader))
                test_losses.append(test_loss/len(testloader))    
                print(f"|  [Epoch: {epoch + 1}, Batch: {i + 1}]   "
                      f"Train loss: {running_loss/len(trainloader):.3f}  |  "
                      f"Test loss: {test_loss/len(testloader):.3f}  |  "
                      f"Test accuracy: {accuracy/len(testloader):.3f}  |")
                running_loss = 0
                model.train()
            
    
    PATH = './models/custom_label_1001.pt'
    torch.save(model.state_dict(), PATH)
    return train_losses, test_losses, PATH

def visualize_train(train_losses, test_losses):
     plt.plot(train_losses, label='Training loss')
     plt.plot(test_losses, label='Test/Validation loss')
     plt.legend(frameon=False)
     plt.show()

print("========================================BEGIN TRAINING=======================================")
train_losses, test_losses, PATH = train_network()
print("=========================================END TRAINING========================================")
visualize_train(train_losses, test_losses)

Net(
  (conv11): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv12): Conv2d(20, 20, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv21): Conv2d(20, 40, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv22): Conv2d(40, 40, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (batch2): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2560, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=73, bias=True)
  (dropconv): Dropout(p=0.2, inplace=False)
  (dropfc): Dropout(p=0.4, inplace=False)
  (soft): Softmax(dim=1)
)
CrossEntropyLoss()
Adam (
Param

c:\users\zacan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


|  [Epoch: 1, Batch: 10]   Train loss: 0.516  |  Test loss: 4.004  |  Test accuracy: 0.020  |
|  [Epoch: 1, Batch: 20]   Train loss: 0.512  |  Test loss: 4.004  |  Test accuracy: 0.028  |
|  [Epoch: 1, Batch: 30]   Train loss: 0.511  |  Test loss: 4.003  |  Test accuracy: 0.046  |
|  [Epoch: 1, Batch: 40]   Train loss: 0.510  |  Test loss: 3.995  |  Test accuracy: 0.173  |
|  [Epoch: 1, Batch: 50]   Train loss: 0.506  |  Test loss: 3.952  |  Test accuracy: 0.183  |
|  [Epoch: 1, Batch: 60]   Train loss: 0.501  |  Test loss: 3.892  |  Test accuracy: 0.198  |
|  [Epoch: 1, Batch: 70]   Train loss: 0.499  |  Test loss: 3.826  |  Test accuracy: 0.294  |
|  [Epoch: 1, Batch: 80]   Train loss: 0.497  |  Test loss: 3.803  |  Test accuracy: 0.299  |
|  [Epoch: 2, Batch: 10]   Train loss: 0.642  |  Test loss: 3.785  |  Test accuracy: 0.308  |
|  [Epoch: 2, Batch: 20]   Train loss: 0.488  |  Test loss: 3.746  |  Test accuracy: 0.351  |
|  [Epoch: 2, Batch: 30]   Train loss: 0.491  |  Test loss: 

|  [Epoch: 11, Batch: 80]   Train loss: 0.462  |  Test loss: 3.437  |  Test accuracy: 0.603  |
|  [Epoch: 12, Batch: 10]   Train loss: 0.605  |  Test loss: 3.435  |  Test accuracy: 0.605  |
|  [Epoch: 12, Batch: 20]   Train loss: 0.462  |  Test loss: 3.413  |  Test accuracy: 0.628  |
|  [Epoch: 12, Batch: 30]   Train loss: 0.462  |  Test loss: 3.399  |  Test accuracy: 0.633  |
|  [Epoch: 12, Batch: 40]   Train loss: 0.466  |  Test loss: 3.383  |  Test accuracy: 0.647  |
|  [Epoch: 12, Batch: 50]   Train loss: 0.463  |  Test loss: 3.405  |  Test accuracy: 0.629  |
|  [Epoch: 12, Batch: 60]   Train loss: 0.466  |  Test loss: 3.389  |  Test accuracy: 0.648  |
|  [Epoch: 12, Batch: 70]   Train loss: 0.470  |  Test loss: 3.387  |  Test accuracy: 0.649  |
|  [Epoch: 12, Batch: 80]   Train loss: 0.469  |  Test loss: 3.379  |  Test accuracy: 0.655  |
|  [Epoch: 13, Batch: 10]   Train loss: 0.603  |  Test loss: 3.423  |  Test accuracy: 0.625  |
|  [Epoch: 13, Batch: 20]   Train loss: 0.464  |  

|  [Epoch: 22, Batch: 70]   Train loss: 0.460  |  Test loss: 3.357  |  Test accuracy: 0.673  |
|  [Epoch: 22, Batch: 80]   Train loss: 0.464  |  Test loss: 3.351  |  Test accuracy: 0.679  |
|  [Epoch: 23, Batch: 10]   Train loss: 0.606  |  Test loss: 3.379  |  Test accuracy: 0.649  |
|  [Epoch: 23, Batch: 20]   Train loss: 0.462  |  Test loss: 3.361  |  Test accuracy: 0.669  |
|  [Epoch: 23, Batch: 30]   Train loss: 0.466  |  Test loss: 3.368  |  Test accuracy: 0.666  |
|  [Epoch: 23, Batch: 40]   Train loss: 0.463  |  Test loss: 3.371  |  Test accuracy: 0.659  |
|  [Epoch: 23, Batch: 50]   Train loss: 0.467  |  Test loss: 3.361  |  Test accuracy: 0.668  |
|  [Epoch: 23, Batch: 60]   Train loss: 0.465  |  Test loss: 3.351  |  Test accuracy: 0.677  |
|  [Epoch: 23, Batch: 70]   Train loss: 0.462  |  Test loss: 3.351  |  Test accuracy: 0.679  |
|  [Epoch: 23, Batch: 80]   Train loss: 0.465  |  Test loss: 3.345  |  Test accuracy: 0.685  |
|  [Epoch: 24, Batch: 10]   Train loss: 0.602  |  

|  [Epoch: 33, Batch: 60]   Train loss: 0.461  |  Test loss: 3.342  |  Test accuracy: 0.686  |
|  [Epoch: 33, Batch: 70]   Train loss: 0.463  |  Test loss: 3.335  |  Test accuracy: 0.693  |
|  [Epoch: 33, Batch: 80]   Train loss: 0.467  |  Test loss: 3.344  |  Test accuracy: 0.684  |
|  [Epoch: 34, Batch: 10]   Train loss: 0.606  |  Test loss: 3.346  |  Test accuracy: 0.684  |
|  [Epoch: 34, Batch: 20]   Train loss: 0.464  |  Test loss: 3.351  |  Test accuracy: 0.680  |
|  [Epoch: 34, Batch: 30]   Train loss: 0.466  |  Test loss: 3.338  |  Test accuracy: 0.693  |
|  [Epoch: 34, Batch: 40]   Train loss: 0.460  |  Test loss: 3.342  |  Test accuracy: 0.688  |
|  [Epoch: 34, Batch: 50]   Train loss: 0.463  |  Test loss: 3.342  |  Test accuracy: 0.689  |
|  [Epoch: 34, Batch: 60]   Train loss: 0.459  |  Test loss: 3.339  |  Test accuracy: 0.692  |
|  [Epoch: 34, Batch: 70]   Train loss: 0.460  |  Test loss: 3.336  |  Test accuracy: 0.694  |
|  [Epoch: 34, Batch: 80]   Train loss: 0.462  |  

|  [Epoch: 44, Batch: 50]   Train loss: 0.463  |  Test loss: 3.335  |  Test accuracy: 0.693  |
|  [Epoch: 44, Batch: 60]   Train loss: 0.461  |  Test loss: 3.341  |  Test accuracy: 0.685  |
|  [Epoch: 44, Batch: 70]   Train loss: 0.460  |  Test loss: 3.330  |  Test accuracy: 0.697  |
|  [Epoch: 44, Batch: 80]   Train loss: 0.465  |  Test loss: 3.328  |  Test accuracy: 0.700  |
|  [Epoch: 45, Batch: 10]   Train loss: 0.604  |  Test loss: 3.338  |  Test accuracy: 0.690  |
|  [Epoch: 45, Batch: 20]   Train loss: 0.460  |  Test loss: 3.328  |  Test accuracy: 0.699  |
|  [Epoch: 45, Batch: 30]   Train loss: 0.462  |  Test loss: 3.336  |  Test accuracy: 0.692  |
|  [Epoch: 45, Batch: 40]   Train loss: 0.462  |  Test loss: 3.335  |  Test accuracy: 0.694  |
|  [Epoch: 45, Batch: 50]   Train loss: 0.463  |  Test loss: 3.335  |  Test accuracy: 0.693  |
|  [Epoch: 45, Batch: 60]   Train loss: 0.464  |  Test loss: 3.334  |  Test accuracy: 0.694  |
|  [Epoch: 45, Batch: 70]   Train loss: 0.464  |  

|  [Epoch: 55, Batch: 40]   Train loss: 0.460  |  Test loss: 3.342  |  Test accuracy: 0.684  |
|  [Epoch: 55, Batch: 50]   Train loss: 0.459  |  Test loss: 3.338  |  Test accuracy: 0.690  |
|  [Epoch: 55, Batch: 60]   Train loss: 0.462  |  Test loss: 3.337  |  Test accuracy: 0.692  |
|  [Epoch: 55, Batch: 70]   Train loss: 0.462  |  Test loss: 3.336  |  Test accuracy: 0.693  |
|  [Epoch: 55, Batch: 80]   Train loss: 0.467  |  Test loss: 3.340  |  Test accuracy: 0.688  |
|  [Epoch: 56, Batch: 10]   Train loss: 0.603  |  Test loss: 3.333  |  Test accuracy: 0.693  |
|  [Epoch: 56, Batch: 20]   Train loss: 0.464  |  Test loss: 3.337  |  Test accuracy: 0.689  |
|  [Epoch: 56, Batch: 30]   Train loss: 0.461  |  Test loss: 3.329  |  Test accuracy: 0.698  |
|  [Epoch: 56, Batch: 40]   Train loss: 0.464  |  Test loss: 3.333  |  Test accuracy: 0.695  |
|  [Epoch: 56, Batch: 50]   Train loss: 0.461  |  Test loss: 3.339  |  Test accuracy: 0.686  |
|  [Epoch: 56, Batch: 60]   Train loss: 0.459  |  

|  [Epoch: 66, Batch: 30]   Train loss: 0.463  |  Test loss: 3.333  |  Test accuracy: 0.696  |
|  [Epoch: 66, Batch: 40]   Train loss: 0.459  |  Test loss: 3.331  |  Test accuracy: 0.697  |
|  [Epoch: 66, Batch: 50]   Train loss: 0.461  |  Test loss: 3.334  |  Test accuracy: 0.694  |
|  [Epoch: 66, Batch: 60]   Train loss: 0.463  |  Test loss: 3.331  |  Test accuracy: 0.697  |
|  [Epoch: 66, Batch: 70]   Train loss: 0.464  |  Test loss: 3.336  |  Test accuracy: 0.692  |
|  [Epoch: 66, Batch: 80]   Train loss: 0.463  |  Test loss: 3.328  |  Test accuracy: 0.700  |
|  [Epoch: 67, Batch: 10]   Train loss: 0.602  |  Test loss: 3.331  |  Test accuracy: 0.696  |
|  [Epoch: 67, Batch: 20]   Train loss: 0.458  |  Test loss: 3.328  |  Test accuracy: 0.700  |
|  [Epoch: 67, Batch: 30]   Train loss: 0.460  |  Test loss: 3.331  |  Test accuracy: 0.697  |
|  [Epoch: 67, Batch: 40]   Train loss: 0.460  |  Test loss: 3.332  |  Test accuracy: 0.695  |
|  [Epoch: 67, Batch: 50]   Train loss: 0.465  |  

|  [Epoch: 77, Batch: 20]   Train loss: 0.463  |  Test loss: 3.329  |  Test accuracy: 0.696  |
|  [Epoch: 77, Batch: 30]   Train loss: 0.460  |  Test loss: 3.328  |  Test accuracy: 0.698  |
|  [Epoch: 77, Batch: 40]   Train loss: 0.464  |  Test loss: 3.329  |  Test accuracy: 0.696  |
|  [Epoch: 77, Batch: 50]   Train loss: 0.466  |  Test loss: 3.334  |  Test accuracy: 0.692  |
|  [Epoch: 77, Batch: 60]   Train loss: 0.463  |  Test loss: 3.332  |  Test accuracy: 0.694  |
|  [Epoch: 77, Batch: 70]   Train loss: 0.463  |  Test loss: 3.333  |  Test accuracy: 0.693  |
|  [Epoch: 77, Batch: 80]   Train loss: 0.466  |  Test loss: 3.332  |  Test accuracy: 0.694  |
|  [Epoch: 78, Batch: 10]   Train loss: 0.599  |  Test loss: 3.331  |  Test accuracy: 0.695  |
|  [Epoch: 78, Batch: 20]   Train loss: 0.463  |  Test loss: 3.332  |  Test accuracy: 0.693  |
|  [Epoch: 78, Batch: 30]   Train loss: 0.461  |  Test loss: 3.330  |  Test accuracy: 0.695  |
|  [Epoch: 78, Batch: 40]   Train loss: 0.464  |  